<a href="https://colab.research.google.com/github/hima4274/skills-introduction-to-github/blob/main/New_Hima_Finance_API_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#Task 1: Fetch Basic Stock Data

import json
import pandas as pd
import json
import requests

api_key= "HMofNlGhyj2A0R3P1Iyvs8aWEE8cUtzY7OXMtSgc"
url= "https://yfapi.net/v6/finance/quote"

# Get user input
symbols = input("Enter stock symbols separated by commas: ").split(',')

querystring = {"symbols": ",".join(symbols)}

headers = {
    'x-api-key': api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)
if response.status_code == 200:
    data = response.json()
    if "quoteResponse" in data and "result" in data["quoteResponse"]:
        quotes = data["quoteResponse"]["result"]
        stock_data = []
        for quote in quotes:
            stock_data.append({
                "Symbol": quote.get("symbol", "N/A"),
                "Name": quote.get("longName", "N/A"),
                "Price": quote.get("regularMarketPrice")
            })

print(stock_data)






Enter stock symbols separated by commas: AAPL,MSFT,TSLA
[{'Symbol': 'AAPL', 'Name': 'Apple Inc.', 'Price': 240.36}, {'Symbol': 'MSFT', 'Name': 'Microsoft Corporation', 'Price': 399.73}, {'Symbol': 'TSLA', 'Name': 'Tesla, Inc.', 'Price': 290.8}]


In [14]:
#Task 2.1: Additional Data with Modules

import pandas as pd
import requests

api_key = "HMofNlGhyj2A0R3P1Iyvs8aWEE8cUtzY7OXMtSgc"

def additional_data(symbol):
    url = "https://yfapi.net/v11/finance/quoteSummary/{symbol}".format(symbol=symbol)
    headers = {"x-api-key": api_key}
    params = {"modules": "summaryDetail,financialData"}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print("Error fetching data:", response.status_code)
        return

    data = response.json()

    #get rid of unneccesary info
    quote_data = data.get("quoteSummary", {}).get("result", [{}])[0]

    if not quote_data:
        print("Invalid response format")
        return

    summary_detail = quote_data.get("summaryDetail", {})
    financial_data = quote_data.get("financialData", {})


    stock_data = {
        "Stock Ticker": [symbol.upper()],
        "52 Week High": [summary_detail.get("fiftyTwoWeekHigh", {}).get("raw", "N/A")],
        "52 Week Low": [summary_detail.get("fiftyTwoWeekLow", {}).get("raw", "N/A")],
        "Return on Assets (ROA)": [financial_data.get("returnOnAssets", {}).get("raw", "N/A")]
    }

    stock_df = pd.DataFrame(stock_data)
    print(stock_df)

# prompt User input
symbol = input("Enter a stock symbol: ").strip()
additional_data(symbol)


Enter a stock symbol: AAPL
  Stock Ticker  52 Week High  52 Week Low  Return on Assets (ROA)
0         AAPL         260.1       164.08                 0.22519


In [1]:
#Task 2.2: continued

import requests
import pandas as pd
from IPython.display import display

#background info needed (API)
api_key = "HMofNlGhyj2A0R3P1Iyvs8aWEE8cUtzY7OXMtSgc"
url_for_trends = "https://yfapi.net/v1/finance/trending/US"
endpoint_url= "https://yfapi.net/v11/finance/quoteSummary/{symbol}"

# Function to fetch trending stock tickers
def get_trending_stocks():
    headers = {"x-api-key": api_key}
    response = requests.get(url_for_trends, headers=headers)

    if response.status_code != 200:
        print("Error fetching trending stocks:", response.status_code)
        return []

    data = response.json()
    trending = data.get("finance", {}).get("result", [{}])[0].get("quotes", [])

    return [stock["symbol"] for stock in trending]


def stock_details(symbol):
    url = endpoint_url.format(symbol=symbol)
    headers = {"x-api-key": api_key}
    params = {"modules": "price,summaryDetail"}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error fetching data for {symbol}: {response.status_code}")
        return None

    data = response.json()
    quote_data = data.get("quoteSummary", {}).get("result", [{}])[0]

    if not quote_data:
        return None

    price_info = quote_data.get("price", {})
    summary_detail = quote_data.get("summaryDetail", {})

    return {
        "Ticker": symbol,
        "Company": price_info.get("shortName", "N/A"),
        "Current Price": price_info.get("regularMarketPrice", {}).get("raw", "N/A"),
        "52 Week High": summary_detail.get("fiftyTwoWeekHigh", {}).get("raw", "N/A"),
        "52 Week Low": summary_detail.get("fiftyTwoWeekLow", {}).get("raw", "N/A")
    }


trending_tickers = get_trending_stocks()


stock_data = [stock_details(symbol) for symbol in trending_tickers if stock_details(symbol)]


stock_df = pd.DataFrame(stock_data)
display(stock_df)


,Ticker,Company,Current Price,52 Week High,52 Week Low
0,NVDA,NVIDIA Corporation,131.28,153.130,75.606
1,SNOW,Snowflake Inc.,166.19,233.880,107.130
2,CRM,"Salesforce, Inc.",307.33,369.000,212.000
3,IONQ,"IonQ, Inc.",29.93,54.740,6.220
4,BYND,"Beyond Meat, Inc.",3.56,11.700,3.300
5,MRNA,"Moderna, Inc.",33.58,170.470,29.250
6,MARA,"MARA Holdings, Inc.",12.45,30.710,12.050
7,TDOC,"Teladoc Health, Inc.",10.99,15.950,6.760
8,BTC-USD,Bitcoin USD,84740.07,109114.880,49121.240
9,AI,"C3.ai, Inc.",26.44,45.080,18.850
